In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [3]:
data = pd.read_excel('_cotacoes_completa_diario.xlsx')

In [4]:
data.head(3)

,time,open IBOV,high IBOV,low IBOV,close IBOV,volume IBOV,open ABEV3,high ABEV3,low ABEV3,close ABEV3,...,open WEGE3,high WEGE3,low WEGE3,close WEGE3,volume WEGE3,open YDUQ3,high YDUQ3,low YDUQ3,close YDUQ3,volume YDUQ3
0,2011-07-01,62404,63455,62148,63394,5519544440,7.92,8.03,7.78,8.01,...,4.32,4.39,4.32,4.38,331056,5.06,5.14,4.97,5.14,770164
1,2011-07-04,63397,63891,63314,63891,1848898245,8.01,8.07,7.95,7.99,...,4.35,4.44,4.34,4.34,199471,5.14,5.29,5.06,5.11,270019
2,2011-07-05,63886,63886,63030,63039,3691016029,7.95,8.11,7.92,8.07,...,4.40,4.43,4.37,4.41,1062133,5.19,5.28,5.14,5.20,488745


In [5]:
filter_col = [col for col in data if col.startswith('close')]
dados = data[filter_col].copy()

In [6]:
dados.head(3)

,close IBOV,close ABEV3,close B3SA3,close BBAS3,close BBDC3,close BBDC4,close BRAP4,close BRFS3,close BRKM5,close BRML3,...,close RENT3,close SBSP3,close SUZB3,close TIMP3,close UGPA3,close USIM5,close VALE3,close VIVT4,close WEGE3,close YDUQ3
0,63394,8.01,7.80,16.48,8.25,9.58,25.91,24.43,15.97,10.28,...,6.94,12.32,9.64,7.20,11.35,13.06,35.87,24.98,4.38,5.14
1,63891,7.99,7.75,16.66,8.30,9.73,26.23,24.39,15.81,10.10,...,6.89,12.40,9.73,7.10,11.53,13.35,36.32,24.88,4.34,5.11
2,63039,8.07,7.57,16.58,8.32,9.69,25.85,24.34,15.63,10.12,...,6.71,12.54,9.64,7.09,11.43,13.13,35.60,24.34,4.41,5.20


In [7]:
rsi_period = 14
l_name = []
banana = dados.copy()
for i in dados:
    a = str(i).split()[1]
    chg = dados[i].diff(1) #variação em 1 vela
    gain = chg.mask(chg<0,0) #pega somente as changes positivas
    loss = chg.mask(chg>0,0) #pega somente as changes negativas
#     dados['gain'] = gain
#     dados['loss'] = loss
    avg_gain = gain.ewm(com = rsi_period - 1, min_periods = rsi_period).mean() #exponential weighted function
    avg_loss = loss.ewm(com = rsi_period - 1, min_periods = rsi_period).mean()
    rs = abs(avg_gain/avg_loss)
    rsi = 100-(100/(1+rs))
    dados['RSI '+a] = rsi

In [10]:
valor_carteira = 0

#posicao 1 - comprado,
#0 - neutro, 
#(-1) - vendido

#seleciona os tickers com 'Close'
for ticker in dados.iloc[:,:54]:
    a = (ticker.split()[1])
    rsi_ticker = ('RSI '+a)
    valor_compra = 0
    posicao = 0
    valor_stock = 0 
    valor_compra = 0
    
#     print('\n\n',ticker)
    for i in dados[rsi_ticker].index:
        rsi = dados[rsi_ticker][dados[rsi_ticker].index == i].values[0]
        valor_stock = dados[ticker][dados[ticker].index == i].values[0]
        #comprar
        if rsi<35:
            #se está neutro ou vendido 
            if posicao == 0 or posicao == -1:
                valor_carteira -= valor_stock #comprou 1 
                valor_compra = valor_stock
                posicao +=1
                print(f'comprou a: {valor_stock}')
        
#         #zerar
#         elif rsi >= 47 and rsi <= 54:
#             #se tiver comprado
#             if posicao == 1: 
#                 valor_carteira += valor_stock #vendeu 1 
#                 posicao -=1
#                 print(f'vendeu a : {valor_stock}')
#                 print(f'resultado: {valor_carteira}\n\n')
                      
        #zerar
        elif valor_stock - valor_compra >= .01*valor_compra:
            #se tiver comprado
            if posicao == 1: 
                valor_carteira += valor_stock #vendeu 1 
                posicao -=1
                print(f'vendeu a : {valor_stock}')
                print(f'resultado: R${valor_carteira}\n\n')

comprou a: 59340
vendeu a : 59962
resultado: R$622


comprou a: 62923
vendeu a : 63782
resultado: R$1481


comprou a: 57111
vendeu a : 57937
resultado: R$2307


comprou a: 61597
vendeu a : 62639
resultado: R$3349


comprou a: 72415
vendeu a : 74363
resultado: R$5297


comprou a: 70827
vendeu a : 72512
resultado: R$6982


comprou a: 80122
vendeu a : 81435
resultado: R$8295


comprou a: 90024
vendeu a : 91946
resultado: R$10217


comprou a: 105718
vendeu a : 107224
resultado: R$11723


comprou a: 102233
comprou a: 7.49
vendeu a : 7.61
resultado: R$-90509.88


comprou a: 11.65
vendeu a : 11.98
resultado: R$-90509.55


comprou a: 13.63
vendeu a : 13.77
resultado: R$-90509.41


comprou a: 13.54
vendeu a : 13.95
resultado: R$-90509.0


comprou a: 13.07
vendeu a : 13.25
resultado: R$-90508.82


comprou a: 13.02
vendeu a : 13.58
resultado: R$-90508.26000000001


comprou a: 13.55
vendeu a : 13.97
resultado: R$-90507.84000000001


comprou a: 13.16
vendeu a : 13.51
resultado: R$-90507.49000000002

KeyboardInterrupt: 

In [142]:
datadados.columns

Index(['close IBOV', 'close ABEV3', 'close B3SA3', 'close BBAS3',
       'close BBDC3', 'close BBDC4', 'close BRAP4', 'close BRFS3',
       'close BRKM5', 'close BRML3',
       ...
       'RSI RENT3', 'RSI SBSP3', 'RSI SUZB3', 'RSI TIMP3', 'RSI UGPA3',
       'RSI USIM5', 'RSI VALE3', 'RSI VIVT4', 'RSI WEGE3', 'RSI YDUQ3'],
      dtype='object', length=108)

In [140]:
dados.iloc[:,:54]

,close IBOV,close ABEV3,close B3SA3,close BBAS3,close BBDC3,close BBDC4,close BRAP4,close BRFS3,close BRKM5,close BRML3,...,close RENT3,close SBSP3,close SUZB3,close TIMP3,close UGPA3,close USIM5,close VALE3,close VIVT4,close WEGE3,close YDUQ3
0,63394,8.01,7.80,16.48,8.25,9.58,25.91,24.43,15.97,10.28,...,6.94,12.32,9.64,7.20,11.35,13.06,35.87,24.98,4.38,5.14
1,63891,7.99,7.75,16.66,8.30,9.73,26.23,24.39,15.81,10.10,...,6.89,12.40,9.73,7.10,11.53,13.35,36.32,24.88,4.34,5.11
2,63039,8.07,7.57,16.58,8.32,9.69,25.85,24.34,15.63,10.12,...,6.71,12.54,9.64,7.09,11.43,13.13,35.60,24.34,4.41,5.20
3,62565,8.04,7.51,16.44,8.20,9.57,25.79,23.44,15.38,9.90,...,6.63,12.28,9.31,7.14,11.40,12.96,35.48,24.28,4.58,5.14
4,62207,7.94,7.62,16.00,8.07,9.44,25.95,23.51,15.09,9.85,...,6.79,12.44,9.51,7.19,11.20,12.63,35.70,24.25,4.44,5.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2157,74073,11.86,37.21,27.86,17.65,19.07,28.87,17.68,16.56,9.14,...,27.95,35.33,40.02,13.38,12.43,4.03,43.13,50.55,33.26,21.80
2158,76358,12.41,38.50,29.17,18.63,19.79,29.22,17.53,16.72,9.12,...,29.17,37.27,37.19,13.38,12.87,4.07,43.78,49.81,37.28,26.01
2159,78625,12.31,39.31,29.73,18.96,20.31,29.16,17.40,16.51,9.95,...,31.84,39.28,37.96,13.54,13.72,4.35,43.51,51.02,39.97,26.94
2160,77682,12.10,39.01,29.65,18.75,20.30,28.61,18.04,16.29,9.45,...,30.10,40.62,35.40,13.31,14.24,4.29,43.28,49.39,38.28,26.55


In [41]:
"""
cotacoes = dados[rsi_ticker]
opera(cotacoes)


"""
def opera(dados_rsiticker):
    dic[a] = 0
    
    posicao = 0 #vendido/neutro/comprado
    resultado = 0 
    preco_compra = 0
    carteira = 0
    
    for i in dados_rsiticker.index:
        rsi = dados_rsiticker[dados_rsiticker.index == i].values[0]
        preco_acao = dados[ticker][dados[ticker].index == i].values[0]
        #comprar
        if rsi <= 35:
            if posicao == 0: #só compra se estiver neutro
                posicao +=1 #agora está comprado em 1
                preco_compra = preco_acao #salva o preço de compra
                carteira -= preco_acao #desconta na carteira o valor da ação
#                 print(f'comprou {a} a: {preco_compra}')     
                              
        #zerar
        elif preco_acao - preco_compra >= .1*preco_compra and posicao == 1:
            #se tiver comprado e lucro de 4%
            carteira += preco_acao #credita o valor da ação vendida
            posicao -= 1
            dic[a] += 1
    
    return dic
#             print(f'vendeu a : {preco_acao}')
#             print(f'resultado: R${carteira}\n\n')
    



In [42]:
valor_carteira = 0

#posicao 1 - comprado,
#0 - neutro, 
#(-1) - vendido

#seleciona os tickers com 'Close'
for ticker in dados.iloc[:,:54]:
    a = (ticker.split()[1])
    rsi_ticker = ('RSI '+a)
    cotacoes = dados[rsi_ticker]
    opera(cotacoes) #executa função para compra e venda 

In [47]:
total_operacoes = 0
for k,v in dic.items():
    print(f'{k}: {v}')
    total_operacoes += v
    
print(total_operacoes)

BRFS3: 7
IBOV: 5
ABEV3: 8
B3SA3: 15
BBAS3: 16
BBDC3: 16
BBDC4: 16
BRAP4: 6
BRKM5: 7
BRML3: 3
BTOW3: 8
CCRO3: 11
CIEL3: 6
CMIG4: 5
CSAN3: 8
CSNA3: 2
CYRE3: 8
ECOR3: 4
EGIE3: 8
ELET3: 5
ELET6: 5
EMBR3: 9
ENBR3: 9
EQTL3: 11
FLRY3: 5
GGBR4: 5
GOAU4: 2
GOLL4: 4
HYPE3: 10
IGTA3: 11
ITSA4: 12
ITUB4: 13
JBSS3: 8
LAME4: 9
LREN3: 10
MGLU3: 3
MRFG3: 0
MRVE3: 19
MULT3: 8
PETR3: 7
PETR4: 11
QUAL3: 13
RADL3: 11
RAIL3: 0
RENT3: 8
SBSP3: 9
SUZB3: 9
TIMP3: 9
UGPA3: 9
USIM5: 5
VALE3: 5
VIVT4: 9
WEGE3: 13
YDUQ3: 7
442
